1. Importing Libraries and Setting Up Variables

In [1]:
import pandas as pd
import psycopg2
import json

This block imports the necessary libraries: pandas for data manipulation, psycopg2 for PostgreSQL database connection, and json for reading the database configuration file. It then reads the CSV file candidates.csv into a DataFrame df

In [2]:
csv ="candidates.csv"
df= pd.read_csv(csv, delimiter = ";")

2. DataFrame Information

In [3]:
df.info


<bound method DataFrame.info of        First Name   Last Name                       Email Application Date  \
0      Bernadette   Langworth         leonard91@yahoo.com       2021-02-26   
1          Camryn    Reynolds         zelda56@hotmail.com       2021-09-09   
2           Larue      Spinka    okey_schultz41@gmail.com       2020-04-14   
3            Arch      Spinka      elvera_kulas@yahoo.com       2020-10-01   
4           Larue  Altenwerth   minnie.gislason@gmail.com       2020-05-20   
...           ...         ...                         ...              ...   
49995     Bethany     Shields  rocky_mitchell@hotmail.com       2022-01-09   
49996         Era  Swaniawski    dolores.roob@hotmail.com       2020-06-02   
49997      Martin       Lakin   savanah.stracke@gmail.com       2018-12-15   
49998       Aliya   Abernathy  vivienne.fritsch@yahoo.com       2020-05-30   
49999     Coleman      Wisozk   abigayle.crooks@yahoo.com       2022-06-13   

                  Country  YOE 

In [4]:
df.isnull().sum()

First Name                   0
Last Name                    0
Email                        0
Application Date             0
Country                      0
YOE                          0
Seniority                    0
Technology                   0
Code Challenge Score         0
Technical Interview Score    0
dtype: int64

In [5]:
df.columns

Index(['First Name', 'Last Name', 'Email', 'Application Date', 'Country',
       'YOE', 'Seniority', 'Technology', 'Code Challenge Score',
       'Technical Interview Score'],
      dtype='object')

In [6]:
df.dtypes

First Name                   object
Last Name                    object
Email                        object
Application Date             object
Country                      object
YOE                           int64
Seniority                    object
Technology                   object
Code Challenge Score          int64
Technical Interview Score     int64
dtype: object

In [9]:


#Function to read the connection configuration from a JSON file
def read_db_config(json_file_path):
    with open(json_file_path, 'r') as file:
        return json.load(file)

configuration = read_db_config('finaldatabase.json')

#Function to connect to an existing database
def connect_to_database(json_file_path):
    try:
        configuration = read_db_config(json_file_path)

        # Establish the connection to the database
        connection = psycopg2.connect(
            user=configuration['user'],
            password=configuration['password'],
            database=configuration['dbname'] 
        )

        print("Database connection successfully established.")
        return connection

    except Exception as e:
        print(f"Failed to connect to the database: {e}")
        return None

#Connect to the database
connection = connect_to_database('finaldatabase.json')

#function to create the candidates table
def create_candidates_table(connection):
    # Create a new cursor
    cursor = connection.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Candidates (
            CandidateID SERIAL PRIMARY KEY,
            FirstName VARCHAR(255) NOT NULL,
            LastName VARCHAR(255) NOT NULL,
            Email VARCHAR(255) NOT NULL,
            ApplicationDate DATE NOT NULL,
            Country VARCHAR(255) NOT NULL,
            Experience INT NOT NULL,
            SeniorityLevel VARCHAR(255) NOT NULL,
            TechnologyStack VARCHAR(255) NOT NULL,
            CodeChallengeScore SMALLINT NOT NULL,
            TechnicalInterviewScore SMALLINT NOT NULL,
            Hired BOOLEAN NOT NULL
        );
    """)
    
    #Confirm the creation of the table
    connection.commit()
    
    #Close the cursor
    cursor.close()

    print("Candidates table successfully created.")

#Create the table using connection
create_candidates_table(connection)

#Close the connection
connection.close()


Database connection successfully established.
Candidates table successfully created.


In [10]:
csv = "candidates.csv"
df = pd.read_csv(csv, delimiter = ";")

In [11]:
#Function to insert data into the table candidates.
def insert_candidate_data(df, connection):
    cursor = connection.cursor()
    insert_query = '''
        INSERT INTO Candidates (FirstName, LastName, Email, ApplicationDate, Country, Experience, SeniorityLevel, TechnologyStack, CodeChallengeScore, TechnicalInterviewScore, Hired)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    for _, row in df.iterrows():
        hired = row['Code Challenge Score'] >= 7 and row['Technical Interview Score'] >= 7
        data_tuple = (row['First Name'], row['Last Name'], row['Email'], row['Application Date'], row['Country'], row['YOE'], row['Seniority'], row['Technology'], row['Code Challenge Score'], row['Technical Interview Score'], hired)
        cursor.execute(insert_query, data_tuple)
    connection.commit()
    print('Data successfully inserted into the Candidates table.')
    cursor.close()

#Connect to the database
connection = connect_to_database('finaldatabase.json') 
insert_candidate_data(df, connection)  #Call the function to insert data passing the DataFrame
connection.close()  # Close the connection


Database connection successfully established.
Data successfully inserted into the Candidates table.


In [12]:
def fetch_updated_data():
    #conection to database
    connection = connect_to_database('finaldatabase.json')
    
    #use the cursor
    cursor = connection.cursor()
    
    #select candidates
    cursor.execute("SELECT * FROM Candidates;")
    
    #new df
    updated_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    
    
    cursor.close()
    connection.close()
    
    return updated_df

#updated df
updated_df = fetch_updated_data()
print(updated_df.head())


Database connection successfully established.
   candidateid   firstname    lastname                      email  \
0            1  Bernadette   Langworth        leonard91@yahoo.com   
1            2      Camryn    Reynolds        zelda56@hotmail.com   
2            3       Larue      Spinka   okey_schultz41@gmail.com   
3            4        Arch      Spinka     elvera_kulas@yahoo.com   
4            5       Larue  Altenwerth  minnie.gislason@gmail.com   

  applicationdate  country  experience senioritylevel  \
0      2021-02-26   Norway           2         Intern   
1      2021-09-09   Panama          10         Intern   
2      2020-04-14  Belarus           4      Mid-Level   
3      2020-10-01  Eritrea          25        Trainee   
4      2020-05-20  Myanmar          13      Mid-Level   

                     technologystack  codechallengescore  \
0                      Data Engineer                   3   
1                      Data Engineer                   2   
2               

In [13]:
updated_df.info

<bound method DataFrame.info of        candidateid   firstname    lastname                       email  \
0                1  Bernadette   Langworth         leonard91@yahoo.com   
1                2      Camryn    Reynolds         zelda56@hotmail.com   
2                3       Larue      Spinka    okey_schultz41@gmail.com   
3                4        Arch      Spinka      elvera_kulas@yahoo.com   
4                5       Larue  Altenwerth   minnie.gislason@gmail.com   
...            ...         ...         ...                         ...   
49995        49996     Bethany     Shields  rocky_mitchell@hotmail.com   
49996        49997         Era  Swaniawski    dolores.roob@hotmail.com   
49997        49998      Martin       Lakin   savanah.stracke@gmail.com   
49998        49999       Aliya   Abernathy  vivienne.fritsch@yahoo.com   
49999        50000     Coleman      Wisozk   abigayle.crooks@yahoo.com   

      applicationdate             country  experience senioritylevel  \
0      

In [14]:
updated_df.columns

Index(['candidateid', 'firstname', 'lastname', 'email', 'applicationdate',
       'country', 'experience', 'senioritylevel', 'technologystack',
       'codechallengescore', 'technicalinterviewscore', 'hired'],
      dtype='object')

In [ ]:
updated_df.describe()

In [ ]:
grouped_data2 = updated_df.groupby("technicalinterviewscore")
grouped_data2.describe()

In [ ]:
grouped_data1 = updated_df.groupby("hired")
grouped_data1.describe()